### Library

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import itertools
from tqdm import tqdm
import FinanceDataReader as fdr
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.arima.model import ARIMA

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

### Data Path

In [2]:
data_dir = Path('../Stock_Price_Predictor/open/')

stock_list_csv = data_dir / 'stock_list.csv'
sub_csv = data_dir / 'sample_submission.csv'

In [3]:
stock_list = pd.read_csv(stock_list_csv)
sub = pd.read_csv(sub_csv)

In [4]:
stock_list = stock_list['종목코드'].apply(lambda x : str(x).zfill(6))
stock_list

0      005930
1      000660
2      035420
3      035720
4      207940
        ...  
365    220630
366    064260
367    287410
368    110790
369    099320
Name: 종목코드, Length: 370, dtype: object

### Stock Data Reader

In [5]:
START = '20111031'
END = '20211031'

def stock_reader(code):
    stock = fdr.DataReader(code, start = START, end = END)
    return stock['Close']

### ARIMA Model

In [6]:
p = [1,2,4,6,8]
d = q = range(0,3)
pdq = list(itertools.product(p, d, q))

def arima_model(df):
    aic=[]
    params=[]
        
    for i in pdq:
        try:
            model = ARIMA(df.values, 
                    order=i, 
                    enforce_invertibility=False,
                    enforce_stationarity=False)
            model_fit = model.fit()
            aic.append(round(model_fit.aic,2))
            params.append(i)
            
        except: continue
    
    best_aic = [(pdq[i],j) for i,j in enumerate(aic) if j == min(aic)]
    
    model = ARIMA(df.values, order=best_aic[0][0])
    model_fit = model.fit()
    forecast = model_fit.forecast(steps=5)
    
    return forecast

### Main

In [7]:
# for i,st in tqdm(enumerate(stock_list)):
#     print("===========Model {0}===========".format(i), '>>>', st)
#     stock = stock_reader(st)
    
#     forcast = arima_model(stock)
#     forcast = np.ceil(np.array(forcast))
    
#     sub.loc[:4,st] = forcast